In [4]:
import pandas as pd
import numpy as np

import string
import regex as re
from collections import defaultdict
from tqdm import tqdm  # Changed this line
import csv
import json
import pickle

import matplotlib.pyplot as plt
plt.close('all')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 250)

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from math import ceil
import time

import gc
import sys

forbidden_words = ['abbo ', 'abo ', 'abortion', 'abuse', 'addict', 'addicts', 'adult', 'alla ', 'allah ', 'alligatorbait', 'amateur', 'anal ', 'analannie', 'analsex', 'angry', 'anus ', 'areola', 'argie ', 'aroused', 'arse ', 'arsehole', 'asian ', 'ass ', 'assassin', 'assassinate', 'assassination', 'assault', 'assbagger', 'assblaster', 'assclown', 'asscowboy', 'asses ', 'assfuck', 'assfucker', 'asshat', 'asshole', 'assholes', 'asshore', 'assjockey', 'asskiss', 'asskisser', 'assklown', 'asslick', 'asslicker', 'asslover', 'assman ', 'assmonkey', 'assmunch', 'assmuncher', 'asspacker', 'asspirate', 'asspuppies', 'assranger', 'asswhore', 'asswipe', 'athletesfoot', 'attack', 'babe ', 'babies', 'backdoor', 'backdoorman', 'backseat', 'badfuck', 'balllicker', ' balls ', 'ballsack', 'banging', 'baptist', 'barelylegal', 'barf ', 'barface', 'barfface', 'bast ', 'bastard', 'bazongas', 'bazooms', 'beaner', 'beastality', 'beastial', 'beastiality', 'beatoff', 'beat-off', 'beatyourmeat', 'beaver', 'bestial', 'bestiality', 'biatch', 'bicurious', 'bigass', 'bigbastard', 'bigbutt', 'bisexual', 'bi-sexual', 'bitch', 'bitcher', 'bitches', 'bitchez', 'bitchin', 'bitching', 'bitchslap', 'bitchy', 'biteme', 'blackman', 'blackout', 'blacks', 'blind', 'blow ', 'blowjob', 'boang', 'bogan', 'bohunk', 'bollick', 'bollock', 'bombers', 'bombing', 'bombs', 'bomd', 'bondage', 'boner', 'bong', 'boob', 'boobies', 'boobs', 'booby', 'boody', 'boong', 'boonga', 'boonie', 'booty', 'bootycall', 'bountybar', 'bra ', 'brea5t', 'breast', 'breastjob', 'breastlover', 'breastman', 'brothel', 'bugger', 'buggered', 'buggery', 'bullcrap', 'bulldike', 'bulldyke', 'bullshit', 'bumblefuck', 'bumfuck', 'bunga ', 'bunghole', 'buried', 'butchbabes', 'butchdike', 'butchdyke', 'butt ', 'buttbang', 'butt-bang', 'buttface', 'buttfuck', 'butt-fuck', 'buttfucker', 'butt-fucker', 'buttfuckers', 'butt-fuckers', 'butthead', 'buttman', 'buttmunch', 'buttmuncher', 'buttpirate', 'buttplug', 'buttstain', 'byatch', 'cacker', 'cameljockey', 'cameltoe', 'canadian', 'carpetmuncher', 'carruth', 'cemetery', 'chav ', 'cherrypopper', 'chickslick', "children's", 'chinaman', 'chinamen', 'chinese', 'chink ', 'chinky', 'choad', 'chode', 'cigs ', 'clamdigger', 'clamdiver', 'clit ', 'clitoris', 'clogwog', 'cocaine', 'cock ', 'cockblock', 'cockblocker', 'cockcowboy', 'cockfight', 'cockhead', 'cockknob', 'cocklicker', 'cocklover', 'cocknob', 'cockqueen', 'cockrider', 'cocksman', 'cocksmith', 'cocksmoker', 'cocksucer', 'cocksuck', 'cocksucked', 'cocksucker', 'cocksucking', 'cocktail', 'cocktease', 'cocky', 'cohee', 'coitus', 'commie', 'communist', 'condom', 'conservative', 'conspiracy', 'coolie', 'cooly', 'coon ', 'coondog', 'copulate', 'cornhole', 'corruption', 'cra5h', 'crabs', 'crack', 'crackpipe', 'crackwhore', 'crack-whore', 'crap ', 'crapola', 'crapper', 'crappy', 'crash ', 'crotch ', 'crotchjockey', 'crotchmonkey', 'crotchrot', 'cum ', 'cumbubble', 'cumfest', 'cumjockey', 'cumm', 'cummer', 'cumming', 'cumquat', 'cumqueen', 'cumshot', 'cunilingus', 'cunillingus', 'cunn ', 'cunnilingus', 'cunntt', 'cunt ', 'cunteyed', 'cuntfuck', 'cuntfucker', 'cuntlick', 'cuntlicker', 'cuntlicking', 'cuntsucker', 'cybersex', 'cyberslimer', 'dago ', 'dahmer', 'dammit', 'damn ', 'damnation', 'damnit', 'darkie', 'darky', 'datnigga', 'dead ', 'deapthroat', 'death', 'deepthroat', 'defecate', 'dego ', 'demon', 'desire', 'destroy', 'deth ', 'devilworshipper', 'dick ', 'dickbrain', 'dickforbrains', 'dickhead', 'dickless', 'dicklick', 'dicklicker', 'dickman', 'dickwad', 'dickweed', 'diddle', 'dike ', 'dildo', 'dingleberry', 'dink ', 'dipshit', 'dipstick', 'dirty ', 'disease', 'diseases', 'disturbed', 'dix ', 'dixiedike', 'dixiedyke', 'doggiestyle', 'doggystyle', 'dong ', 'doodoo', 'doo-doo', 'doom ', 'dope ', 'dragqueen', 'dragqween', 'dripdick', 'drug ', 'drunk', 'drunken', 'dumb ', 'dumbass', 'dumbbitch', 'dumbfuck', 'dyefly', 'dyke ', 'easyslut', 'eatballs', 'eatpussy', 'ecstacy', 'ejaculate', 'ejaculated', 'ejaculating', 'ejaculation', 'enema', 'enemy', 'erect', 'erection', ' ero ', 'escort', 'ethiopian', 'ethnic', 'european', ' evl ', 'excrement', 'execute ', 'executed', 'execution', 'executioner', 'explosion', 'facefucker', 'faeces', ' fag ', 'fagging', 'faggot', 'fagot', 'failed', 'failure', 'fairies', 'fannyfucker', 'fart ', 'farted', 'farting', 'farty', 'fastfuck', 'fat ', 'fatah', 'fatass', 'fatfuck', 'fatfucker', 'fatso', 'fckcum', 'feces', 'felatio', 'felch', 'felcher', 'felching', 'fellatio', 'feltch', 'feltcher', 'feltching', 'fetish', 'filipina', 'filipino', 'fingerfood', 'fingerfuck', 'fingerfucked', 'fingerfucker', 'fingerfuckers', 'fingerfucking', 'fister', 'fistfuck', 'fistfucked', 'fistfucker', 'fistfucking', 'fisting', 'flange', 'flasher', 'flatulence', 'floo ', 'flydie', 'flydye', 'fok ', 'fondle', 'footaction', 'footfuck', 'footfucker', 'footlicker', 'footstar', 'foreskin', 'forni', 'fornicate', 'foursome', 'fourtwenty', 'fraud', 'freakfuck', 'freakyfucker', 'freefuck', 'fubar', 'fucck', 'fuck', 'fucka', 'fuckable', 'fuckbag', 'fuckbuddy', 'fucked', 'fuckedup', 'fucker', 'fuckers', 'fuckface', 'fuckfest', 'fuckfreak', 'fuckfriend', 'fuckhead', 'fuckher', 'fuckin', 'fuckina', 'fucking', 'fuckingbitch', 'fuckinnuts', 'fuckinright', 'fuckit', 'fuckknob', 'fuckme', 'fuckmehard', 'fuckmonkey', 'fuckoff', 'fuckpig', 'fucks', 'fucktard', 'fuckwhore', 'fuckyou', 'fudgepacker', 'fugly', ' fuk ', 'fuks', 'funeral', 'funfuck', 'fungus', 'fuuck', 'gangbang', 'gangbanged', 'gangbanger', 'gangsta', 'gatorbait', 'gaymuthafuckinwhore', 'gaysex', 'geez', 'geezer', 'geni', 'genital', 'german', 'getiton', 'ginzo', 'gipp', 'girls', 'givehead', 'glazeddonut', 'godammit', 'goddamit', 'goddammit', 'goddamn', 'goddamned', 'goddamnes', 'goddamnit', 'goddamnmuthafucker', 'goldenshower', 'gonorrehea', 'gonzagas', 'gook', 'gotohell', 'greaseball', 'gringo', 'groe ', 'gross ', 'grostulation', 'gubba', 'gummer', 'gypo', 'gypp', 'gyppie', 'gyppo', 'gyppy', 'hamas', 'handjob', 'hapa', 'harder', 'hardon', 'harem', 'headfuck', 'headlights', 'hebe ', 'heeb ', 'henhouse', 'heroin', 'herpes', 'heterosexual', 'hijack', 'hijacker', 'hijacking', 'hillbillies', 'hindoo', 'hiscock', 'hitler', 'hitlerism', 'hitlerist', 'hiv ', 'hobo', 'hodgie', 'hoes', 'holestuffer', 'homicide', 'homo', 'homobangers', 'homosexual', 'honger', 'honk', 'honkers', 'honkey', 'honky', 'hook', 'hooker', 'hookers', 'hooters', 'hore', 'hork', 'horn', 'horney', 'horniest', 'horny', 'horseshit', 'hosejob', 'hoser', 'hostage', 'hotdamn', 'hotpussy', 'hottotrot', 'husky', 'hussy', 'hustler', 'hymen', 'hymie', 'iblowu', 'idiot', 'ikey', 'illegal', 'incest', 'insest', 'intercourse', 'interracial', 'intheass', 'inthebuff', 'jackass', 'jackoff', 'jackshit', 'jacktheripper', 'japcrap', 'jebus', 'jeez', 'jerkoff', 'jesus', 'jesuschrist', 'jiga', 'jigaboo', 'jigg', 'jigga', 'jiggabo', 'jigger', 'jiggy', 'jihad', 'jijjiboo', 'jimfish', 'jism', 'jizim', 'jizjuice', 'jizm', 'jizz', 'jizzim', 'jizzum', 'joint', 'juggalo', 'jugs', 'junglebunny', 'kaffer', 'kaffir', 'kaffre', 'kafir', 'kanake', 'kigger', 'kike', 'kill', 'killed', 'killer', 'killing', 'kills', 'kink', 'kinky', 'kissass', 'knife', 'knockers', 'kock', 'kondum', 'koon', 'kotex', 'krap', 'krappy', 'kraut', 'kum ', 'kumbubble', 'kumbullbe', 'kummer', 'kumming', 'kumquat', 'kums', 'kunilingus', 'kunnilingus', 'kunt', 'kyke', 'lactate', 'laid', 'lapdance', 'latin', 'lesbain', 'lesbayn', 'lesbian', 'lesbin', 'lesbo', 'lezbe', 'lezbefriends', 'lezbo', 'lezz', 'lezzo', 'liberal', 'libido', 'licker', 'lickme', 'limey', 'limpdick', 'limy', 'lingerie', 'liquor', 'livesex', 'loadedgun', 'lolita', 'looser', 'loser', 'lotion', 'lovebone', 'lovegoo', 'lovegun', 'lovejuice', 'lovemuscle', 'lovepistol', 'loverocket', 'lowlife', 'lubejob', 'lucifer', 'luckycammeltoe', 'lugan', 'lynch', 'macaca', 'mafia ', 'magicwand', 'manhater', 'manpaste', 'marijuana', 'mastabate', 'mastabater', 'masterbate', 'masterblaster', 'mastrabator', 'masturbate', 'masturbating', 'mattressprincess', 'meatbeatter', 'meatrack', 'meth ', 'mexican', 'mgger', 'mggor', 'mickeyfinn', 'mideast', 'milf', 'minority', 'mockey', 'mockie', 'mocky', 'mofo', 'moky', 'moles', 'molest', 'molestation', 'molester', 'molestor', 'moneyshot', 'mooncricket', 'mormon', 'moron', 'moslem', 'mosshead', 'mothafuck', 'mothafucka', 'mothafuckaz', 'mothafucked', 'mothafucker', 'mothafuckin', 'mothafucking', 'mothafuckings', 'motherfuck', 'motherfucked', 'motherfucker', 'motherfuckin', 'motherfucking', 'motherfuckings', 'motherlovebone', 'muff', 'muffdive', 'muffdiver', 'muffindiver', 'mufflikcer', 'mulatto', 'muncher', 'munt', 'murder', 'murderer', 'naked', 'narcotic', 'nasty', 'nastybitch', 'nastyho', 'nastyslut', 'nastywhore', 'nazi', 'necro', 'negro', 'negroes', 'negroid', "negro's", 'niger', 'nigerian', 'nigerians', 'nigg', 'nigga', 'niggah', 'niggaracci', 'niggard', 'niggarded', 'niggarding', 'niggardliness', "niggardliness's", 'niggardly', 'niggards', "niggard's", 'niggaz', 'nigger', 'niggerhead', 'niggerhole', 'niggers', "nigger's", 'niggle', 'niggled', 'niggles', 'niggling', 'nigglings', 'niggor', 'niggur', 'niglet', 'nignog', 'nigr', 'nigra', 'nigre', 'nip ', 'nipple', 'nipplering', 'nittit', 'nlgger', 'nlggor', 'nofuckingway', 'nook', 'nookey', 'nookie', 'noonan', 'nooner', 'nude', 'nudger', 'nuke', 'nutfucker', 'nymph', 'ontherag', 'oral', 'orga ', 'orgasim', 'orgasm', 'orgies', 'orgy', 'osama', 'paki', 'palesimian', 'palestinian', 'pansies', 'pansy', 'panti', 'panties', 'payo', 'pearlnecklace', 'peck', 'pecker', 'peckerwood', 'pee ', 'peehole', 'pee-pee', 'peepshow', 'peepshpw', 'pendy', 'penetration', 'peni5', 'penile', 'penis', 'penises', 'penthouse', 'perv', 'phonesex', 'phuk', 'phuked', 'phuking', 'phukked', 'phukking', 'phungky', 'phuq', 'pi55', 'picaninny', 'piccaninny', 'pickaninny', 'piker', 'pikey', 'piky', 'pimp', 'pimped', 'pimper', 'pimpjuic', 'pimpjuice', 'pimpsimp', 'pindick', 'piss', 'pissed', 'pisser', 'pisses', 'pisshead', 'pissin', 'pissing', 'pissoff', 'pistol', 'pixie', 'pixy', 'playboy', 'playgirl', 'pocha', 'pocho', 'pocketpool', 'pohm', 'polack', 'pom', 'pommie', 'pommy', 'poon', 'poontang', 'poop', 'pooper', 'pooperscooper', 'pooping', 'poorwhitetrash', 'popimp', 'porchmonkey', 'porn', 'pornflick', 'pornking', 'porno', 'pornography', 'pornprincess', 'poverty', 'premature', 'prick', 'prickhead', 'primetime', 'propaganda', 'prostitute', 'protestant', 'pu55i', 'pu55y', 'pube', 'pubic', 'pubiclice', 'pud', 'pudboy', 'pudd', 'puddboy', 'puke', 'puntang', 'purinapricness', 'puss', 'pussie', 'pussies', 'pussy', 'pussycat', 'pussyeater', 'pussyfucker', 'pussylicker', 'pussylips', 'pussylover', 'pussypounder', 'pusy', 'quashie', 'queef', 'queer', 'quickie', 'quim', 'ra8s', 'racial', 'racist', 'radical', 'radicals', 'raghead', 'randy', 'rape ', 'raped', 'raper ', 'rapist', 'rearend', 'rearentry', 'rectum', 'redlight', 'redneck', 'reefer', 'reestie', 'refugee', 'reject', 'remains', 'rentafuck', 'republican', 'rere', 'retard', 'retarded', 'ribbed', 'rigger', 'rimjob', 'rimming', 'roach', 'robber', 'roundeye', 'rump', 'russki', 'russkie', 'sadis', 'sadom', 'samckdaddy', 'sandm', 'sandnigger', 'satan', 'scag', 'scallywag', 'scat', 'schlong', 'screw', 'screwyou', 'scrotum', 'scum', 'semen ', 'seppo', 'servant', 'sex', 'sexed', 'sexfarm', 'sexhound', 'sexhouse', 'sexing', 'sexkitten', 'sexpot', 'sexslave', 'sextogo', 'sextoy', 'sextoys', 'sexual', 'sexually', 'sexwhore', 'sexy', 'sexymoma', 'sexy-slim', 'shag', 'shaggin', 'shagging', 'shat', 'shav', 'shawtypimp', 'sheeney', 'shhit', 'shinola', 'shit', 'shitcan', 'shitdick', 'shite', 'shiteater', 'shited', 'shitface', 'shitfaced', 'shitfit', 'shitforbrains', 'shitfuck', 'shitfucker', 'shitfull', 'shithapens', 'shithappens', 'shithead', 'shithouse', 'shiting', 'shitlist', 'shitola', 'shitoutofluck', 'shits', 'shitstain', 'shitted', 'shitter', 'shitting', 'shitty', 'shoot', 'shooting', 'shortfuck', 'showtime', 'sick', 'sissy', 'sixsixsix', 'sixtynine', 'sixtyniner', 'skank', 'skankbitch', 'skankfuck', 'skankwhore', 'skanky', 'skankybitch', 'skankywhore', 'skinflute', 'skum', 'skumbag', 'slant', 'slanteye', 'slapper', 'slaughter', 'slav', 'slave', 'slavedriver', 'sleezebag', 'sleezeball', 'slideitin', 'slime', 'slimeball', 'slimebucket', 'slopehead', 'slopey', 'slopy', 'slut', 'sluts', 'slutt', 'slutting', 'slutty', 'slutwear', 'slutwhore', 'smack', 'smackthemonkey', 'smut', 'snatch', 'snatchpatch', 'snigger', 'sniggered', 'sniggering', 'sniggers', "snigger's", 'sniper', 'snot', 'snowback', 'snownigger', 'sob ', 'sodom', 'sodomise', 'sodomite', 'sodomize', 'sodomy', 'sonofabitch', 'sonofbitch', 'sooty', 'sos ', 'soviet', 'spaghettibender', 'spaghettinigger', 'spank', 'spankthemonkey', 'sperm', 'spermacide', 'spermbag', 'spermhearder', 'spermherder', 'spic', 'spick', 'spig', 'spigotty', 'spik', 'spit', 'spitter', 'splittail', 'spooge', 'spreadeagle', 'spunk', 'spunky', 'squaw', 'stagg', 'stiffy', 'strapon', 'stringer', 'stripclub', 'stroke', 'stroking', 'stupid', 'stupidfuck', 'stupidfucker', 'suck', 'suckdick', 'sucker', 'suckme', 'suckmyass', 'suckmydick', 'suckmytit', 'suckoff', 'suicide', 'swallow', 'swallower', 'swalow', 'swastika', 'sweetness', 'syphilis', 'taboo', 'taff', 'tampon', 'tang', 'tantra', 'tarbaby', 'tard', 'teat', 'terror', 'terrorist', 'teste', 'testicle', 'testicles', 'thicklips', 'thirdeye', 'thirdleg', 'threesome', 'threeway', 'timbernigger', 'tinkle', 'titbitnipply', 'titfuck', 'titfucker', 'titfuckin', 'titjob', 'titlicker', 'titlover', 'tits', 'tittie', 'titties', 'titty', 'toilet', 'tongethruster', 'tongue', 'tonguethrust', 'tonguetramp', 'tortur', 'torture', 'tosser', 'towelhead', 'trailertrash', 'tramp', 'trannie', 'tranny', 'transexual', 'transsexual', 'transvestite', 'triplex', 'trisexual', 'trojan', 'trots', 'tuckahoe', 'tunneloflove', 'turd', 'turnon', 'twat', 'twink', 'twinkie', 'twobitwhore', 'uck ', 'unfuckable', 'upskirt', 'uptheass', 'upthebutt', 'urinary', 'urinate', 'urine', 'usama', 'uterus', 'vagina', 'vaginal', 'vatican', 'vibr', 'vibrater', 'vibrator', 'vietcong', 'violence', 'virgin', 'virginbreaker', 'vomit', 'vulva', 'wank', 'wanker', 'wanking', 'waysted', 'weapon', 'weenie', 'weewee', 'welcher', 'welfare', 'wetb', 'wetback', 'wetspot', 'whacker', 'whash', 'whigger', 'whiskey', 'whiskeydick', 'whiskydick', 'whitenigger', 'whites', 'whitetrash', 'whitey', 'whiz', 'whop', 'whore', 'whorefucker', 'whorehouse', 'wigger', 'willie', 'williewanker', 'willy', "women's", 'wuss', 'wuzzie', 'xxx', 'yankee', 'yellowman', 'zigabo', 'zipperhead'] # forbidden by Google

def append_trash(trash_ngrams, trash_words_in_ngrams): # IDEA - short numbers are personal data and have no sence in covering
    trash_nums = []
    nums = []

    for i in range(0, 10):
        nums.append(str(i))
        nums.append('0'+str(i))

    for i in range(10, 32):
        nums.append(str(i))

    for num in nums:
        trash_nums.append(' '+str(num))
        trash_nums.append(str(num)+' ')

    #print(trash_nums)

    trash_words_in_ngrams += trash_nums
    trash_ngrams += nums

    for num in nums:
        trash_ngrams.append(str(num)+',')
        trash_ngrams.append(str(num)+'.')

    print('trash_words_in_ngrams: ', trash_words_in_ngrams)
    print('trash_ngrams: ', trash_ngrams)

    return trash_ngrams, trash_words_in_ngrams

def read_and_fix_df(path, x_test=False):
    if path[-5:] == '.json':
        df = pd.read_json(path)
    else:
        df = pd.read_csv(path)

    if x_test: df = df.head(10000)

    print(df['type'].value_counts())

    if 'link_id' not in df.columns:
        df['link_id'] = df['link'].apply(lambda x: re.findall(r"https://my.wisepal.com/hapdne/mainapp/emailmessage/*([^/\s\"]+)", x)[0])

    if 'context_type' not in df.columns:
        df['context_type'] = 'email'

    return df

def extract_email_info(email):
    subject_regex = r"^(.*?)(?= from:)"
    from_section_regex = r"from: (.*?)(?= to:)"
    email_regex = r"([^\s]+@[^\s]+)"
    to_regex = r"to: (?:.*? )?([^\s]+@[^\s]+)(?= \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    date_time_regex = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    body_regex = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} (.*)"

    subject = re.search(subject_regex, email, re.MULTILINE)
    subject = subject.group(1).strip() if subject else None

    from_section = re.search(from_section_regex, email, re.MULTILINE)
    from_section = from_section.group(1).strip() if from_section else ''

    from_email = re.search(email_regex, from_section)
    from_email = from_email.group(1).strip() if from_email else None
    from_email = from_email.replace('<', '').replace('>', '').replace(' ', '') if from_email else None

    sender_name = None
    if from_email:
        sender_name = from_section.replace(from_email, '').strip()

    to_section = re.search(to_regex, email, re.MULTILINE)
    to_section = to_section.group(1).strip() if to_section else ''

    to_email = re.search(email_regex, to_section)
    to_email = to_email.group(1).strip() if to_email else None
    to_email = to_email.replace('<', '').replace('>', '').replace(' ', '') if to_email else None

    date_time = re.search(date_time_regex, email)
    date_time = date_time.group(1).strip() if date_time else None
    body = re.search(body_regex, email, re.MULTILINE | re.DOTALL)
    body = body.group(1).strip() if body else None

    return {'subject': subject, 'sender_name': sender_name, 'from_mail': from_email, 'to_email': to_email, 'date_time': date_time, 'body': body}

def clean_text(text):
    # features of Google API: any symbol except @, &, _, dot in number are supposed to be treated as space and 's too
    text = text.replace('-', ' ')
    text = re.sub(r'(?<!\d)[.,](?!\d)', ' ', text)
    text = text.replace("'s", ' ').replace("’s", ' ')
    text = re.sub(r'[^a-zA-Z0-9.,&]', ' ', text)

    text = re.sub(r'\s{2,}', ' ', text)
    return text

def generate_ngrams(text, n, n_grams):
    email_entities = extract_email_info(text)
    ngrams = []
    if email_entities['subject']:
        words_subject = clean_text(email_entities['subject']).split()
        if n <= len(words_subject):
            ngrams +=  [' '.join(words_subject[i:i + n]) for i in range(len(words_subject) - n + 1)]
    if email_entities['sender_name']:
        words_sender_name = clean_text(email_entities['sender_name']).split()
        if n <= len(words_sender_name):
            ngrams += [' '.join(words_sender_name[i:i + n]) for i in range(len(words_sender_name) - n + 1)]
    if email_entities['from_mail']:
        from_mail = clean_text(email_entities['from_mail']).split()
        if n <= len(from_mail):
            ngrams += [' '.join(from_mail[i:i + n]) for i in range(len(from_mail) - n + 1)]
    if email_entities['body']:
        words_body = clean_text(email_entities['body']).split()
        if n <= len(words_body):
            ngrams += [' '.join(words_body[i:i + n]) for i in range(len(words_body) - n + 1)]
    if email_entities['from_mail'] and n == n_grams[0]:
        ngrams += [email_entities['from_mail']]
    if email_entities['to_email'] and n == n_grams[0]:
        ngrams += [email_entities['to_email']]
    return ngrams

def generate_ngrams_pdf(text, n, n_grams):
    ngrams = []
    if text:
        text = text.replace('\n', ' ')
        words_body = clean_text(text).split()
        if n <= len(words_body):
            ngrams += [' '.join(words_body[i:i + n]) for i in range(len(words_body) - n + 1)]
    return ngrams

def count_total_amount_clusters(df):
    clusters_stats = df.groupby('type').agg(
        count=('context', 'count')).sort_values('count', ascending=False)
    total_amount_clusters = clusters_stats['count'].to_dict()
    total_amount_clusters['good_types'] = len(df.loc[(df['type'] == 'good')].groupby(['link_id']).count())
    total_amount_clusters['bad_types'] = len(df.loc[(df['type'] == 'bad')].groupby(['link_id']).count())
    total_amount_clusters['neutral_types'] = len(df.loc[(df['type'] == 'neutral')].groupby(['link_id']).count())

    return total_amount_clusters

# algorithm Step 1: make 2 times for good and bad
# make dict by ngrams: ngram, count in good/bad, set in good/bad
# filter border good/bad, remove forbidden, remove alone letters - make (1)Good dict and (2)Bad dict
def dict_by_clusters(df, total_amount_clusters, n_grams, good_types, bad_types, neutral_types,
                     trash_ngrams, trash_words_in_ngrams, forbidden_words, ngrams_in_pdf,
                     border_good_cover, border_bad_cover
                     ):
    df = df.copy() # save original df
    df = df.sample(frac=1, random_state=0).reset_index(drop=True)

    df['good_types'] = df['type'].isin(good_types)
    df['bad_types'] = df['type'].isin(bad_types)
    df['neutral_types'] = df['type'].isin(neutral_types)
    cluster_unique = ['good_types_count', 'bad_types_count' , 'neutral_types_count'] #+ df['type'].unique().tolist()
    clusters_count = defaultdict(lambda: {**{x: 0 for x in cluster_unique}, **{x: set() for x in ['good_set', 'bad_set', 'neutral_set']}})

    progress_bar = tqdm(range(len(df)))
    b1 = '\033[1m'
    b2 = '\033[0m'

    for i, row in df.iterrows(): # loop by emails
        for n in range(n_grams[0], n_grams[1] + 1): # range by ngram size
            text = row['context'] # email full text
            type_label = row['type']  # email type
            if row['context_type'] == 'email':
                ngrams = generate_ngrams(text, n, n_grams) # make ngrams by email
            else:
                ngrams = generate_ngrams_pdf(text, n, n_grams)
            # work with img-alt
            if 'img_alts' in df.columns:
                if row['img_alts']:
                    img_alts = list(row['img_alts'].replace("'['","").replace("']'","").split("', '"))
                    for img_alt in img_alts:
                        words_img_alt = clean_text(img_alt).split()
                        if n <= len(words_img_alt):
                            ngrams +=  [' '.join(words_img_alt[i:i + n]) for i in range(len(words_img_alt) - n + 1)]

            #ind = row.name
            ind = row['link_id']
            for ngram in ngrams: # loop by ngrams
                if ngram in trash_ngrams or any([x in ngram for x in trash_words_in_ngrams]): continue
                if type_label in good_types:
                    clusters_count[ngram]['good_types_count'] += 1 # word frequency: how many times the word appeared in good letters
                    clusters_count[ngram]['good_set'].add(ind)
                elif type_label in bad_types:
                    clusters_count[ngram]['bad_types_count'] += 1
                    clusters_count[ngram]['bad_set'].add(ind)
                elif type_label in neutral_types:
                    clusters_count[ngram]['neutral_types_count'] += 1
                    clusters_count[ngram]['neutral_set'].add(ind)
#                clusters_count[ngram][type_label] += 1

        if (i > 0 and i % 10000 == 0) or (i == len(df) - 1):
            #temp_dict.clear()
            temp_dict = {ngram: data for ngram, data in clusters_count.items()
                if len(clusters_count[ngram]['good_set']) / total_amount_clusters['good_types'] >= border_good_cover * (i / len(df))
                or len(clusters_count[ngram]['bad_set']) / total_amount_clusters['bad_types'] >= border_bad_cover * (i / len(df))
            }

            print(f"Iteration: {i}, clusters_count size: {len(clusters_count)}, temp_dict size: {len(temp_dict)}")
            clusters_count_size = sys.getsizeof(clusters_count)
            for k, v in clusters_count.items():
                for set_name in ['good_set', 'bad_set', 'neutral_set']:
                    clusters_count_size += sys.getsizeof(v[set_name])

            temp_dict_size = sys.getsizeof(temp_dict)
            for k, v in temp_dict.items():
                for set_name in ['good_set', 'bad_set', 'neutral_set']:
                    temp_dict_size += sys.getsizeof(v[set_name])

            print(f"Size in RAM - clusters_count: {clusters_count_size / (2**30)} Gb, temp_dict: {temp_dict_size / (2**30)} Gb")

            for k, v in clusters_count.items():
                if k not in temp_dict.keys():
                    v['good_set'] = None
                    v['bad_set'] = None
                    v['neutral_set'] = None
                    #gc.collect()
                    v['good_set'] = set()
                    v['bad_set'] = set()
                    v['neutral_set'] = set()
            clusters_count = None
            #clusters_count.clear()
            gc.collect()
            clusters_count = defaultdict(lambda: {**{x: 0 for x in cluster_unique}, **{x: set() for x in ['good_set', 'bad_set', 'neutral_set']}}, temp_dict)

#            with open(r'/content/drive/MyDrive/ssn_temp_dict v2.pkl', 'wb') as f:
#                pickle.dump(temp_dict, f)

        progress_bar.update(1)

    progress_bar = tqdm(range(len(clusters_count)))
    for ngram in clusters_count:
        clusters_count[ngram]['good_types_len'] = len(clusters_count[ngram]['good_set']) # покрытие слова: в каком количестве писем встречается слово
        clusters_count[ngram]['bad_types_len'] = len(clusters_count[ngram]['bad_set']) # покрытие слова: в каком количестве писем встречается слово
        clusters_count[ngram]['neutral_types_len'] = len(clusters_count[ngram]['neutral_set']) # покрытие слова: в каком количестве писем встречается слово
#        clusters_count[ngram]['good_bad_len_dif'] = clusters_count[ngram]['good_types_len'] - clusters_count[ngram]['bad_types_len'] # полезность слова
        clusters_count[ngram]['good_types_cover'] = clusters_count[ngram]['good_types_len'] / total_amount_clusters['good_types'] # покрытие слова %
        clusters_count[ngram]['bad_types_cover'] = clusters_count[ngram]['bad_types_len'] / total_amount_clusters['bad_types'] # покрытие слова %
#        clusters_count[ngram]['neutral_types_cover'] = clusters_count[ngram]['neutral_types_len'] / total_amount_clusters['neutral_types'] # покрытие слова %
        progress_bar.update(1)

    calc_good = {ngram: data for ngram, data in clusters_count.items()
      if data['good_types_cover'] >= border_good_cover
        and len(ngram) > 2
        and all([x not in f' {ngram} ' for x in forbidden_words])
        }
    # list of -words
    calc_bad = {ngram: data for ngram, data in clusters_count.items()
      if data['bad_types_cover'] >= border_bad_cover
        and data['good_types_len'] == 0
        and data['neutral_types_len'] == 0
        and len(ngram) > 2
        and all([x not in f' {ngram} ' for x in forbidden_words])
        and all([x not in f' {ngram} ' for x in ngrams_in_pdf])
        }

    for k, v in clusters_count.items():
        if k not in calc_good.keys():
            if k not in calc_bad.keys():
                v['good_set'] = None
                v['bad_set'] = None
                v['neutral_set'] = None
    clusters_count = None
    #clusters_count.clear()
    gc.collect()

    return calc_good, calc_bad

def user_unreached(df, union_set, cur_set):
    # list of unreached user/company pairs
    if 'company' in df.columns and 'importance' in df.columns:
        user = df.loc[df['link_id'].isin(union_set)].groupby(['user_id', 'company']).count()
        user_dict = user['importance'].to_dict()

        user_reached = df.loc[df['link_id'].isin(cur_set)].groupby(['user_id', 'company']).count()
        user_r_dict = user_reached['importance'].to_dict()
    else:
        user = df.loc[df['link_id'].isin(union_set)].groupby(['user_id']).count()
        user_dict = user['context'].to_dict()

        user_reached = df.loc[df['link_id'].isin(cur_set)].groupby(['user_id']).count()
        user_r_dict = user_reached['context'].to_dict()

    for k in user_r_dict.keys():
        del user_dict[k]

    return user_dict

#02.04.2024+07.04.2024
# hypothesis - the weight of an email is inversely proportional to the average coverage of its included ngrams
# IDEA TF-IDF like - we first strive to select ngrams that cover as many rare emails as possible on average
def email_ranking(df, calc, set_name='good_set'):
    calc = calc.copy()
    timer = defaultdict(int)

    union_set = set.union(*(value[set_name] for value in calc.values()))

    email_rank = defaultdict(list)
    tic = time.time()
    for ngram_i, value_i in calc.items():
        for email in value_i[set_name]:
            # for each email I change its ngrams to the quantitative coverage of this ngram
            email_rank[email].append(len(value_i[set_name]))
    timer['email_rank'] += time.time()-tic

    # recalculate the inverse of the mean - if the in mean the email has rare ngrams = higher score
    if 'importance' in df.columns:
        email_rank = {k: np.mean(v) for k, v in email_rank.items()}
        # normalization before multiplication
        min_rank = min(email_rank.values())
        max_rank = max(email_rank.values())
        min_imp = df.loc[((df['type'] == 'good'))]['importance'].min()
        max_imp = df.loc[((df['type'] == 'good'))]['importance'].max()
        email_rank = {k: 1 - (((v-min_rank)/(max_rank-min_rank)) * ((df.loc[(df.index==k)]['importance'].values[0]-min_imp)/(max_imp-min_imp)))
                             for k, v in email_rank.items()}
    else:
        email_rank = {k: np.mean(v) for k, v in email_rank.items()}
        min_rank = min(email_rank.values())
        max_rank = max(email_rank.values())
        email_rank = {k: 1 - ((v-min_rank)/(max_rank-min_rank)) for k, v in email_rank.items()}

    return email_rank

# 24.03.2024 minus words
def dict_minus_ngram(df, calc_bad, total_amount_clusters, total_bad_set, max_len=500, min_diff=0.005, x_email_rank=False, email_only=False):
    calc_bad = calc_bad.copy()
    ### idea - only bad email adresses and domains - for more safety
    if email_only:
        calc_bad = {ngram: data for ngram, data in calc_bad.items()
        if '@' in ngram or ngram[-4:] == ' com'
        }
    # only minus words to the remaining bad coverage
    for ngram, data in calc_bad.items():
        data['bad_types_cover'] = len(data['bad_set'] & total_bad_set) / total_amount_clusters['bad_types'] # word cover %

    calc_bad = {ngram: data for ngram, data in calc_bad.items()
      if len(data['bad_set'] & total_bad_set) > 0
      and len(data['good_set']) == 0
    }

    calc_bad = {k: v for k, v in sorted(calc_bad.items(), key=lambda x: (x[1]['bad_types_cover'], -len(x[0])), reverse=True)}

    all_ngrams = list(calc_bad.keys())
    all_ngrams.pop(0)

    cur_diff = 0
    curr = {
        'ngram_list': [next(iter(calc_bad))],
        'bad_set': (set() | set(calc_bad[next(iter(calc_bad))]['bad_set'])),
        }
    print(f"{curr['ngram_list']}\n Bad: {round(len(curr['bad_set']) / len(total_bad_set) * 100, 1)}%; Dif. in bads: {round(len(curr['bad_set']) / len(total_bad_set) * 100, 1)}%; Length: {len('   '.join(curr['ngram_list']))}")

    if x_email_rank: email_rank_bad = email_ranking(df, calc_bad, set_name='bad_set')

    progress_bar = tqdm(range(0, len(total_bad_set)))
    progress_bar.update(len(curr['bad_set']))
    while len(all_ngrams) > 0:
        curr_bad_perc = len(curr['ngram_list']) / len(total_bad_set)
        cur_diff = 0
        if x_email_rank: progress_bar_ngram = tqdm(range(0, len(all_ngrams)))
        for new_ngram, new_data in calc_bad.items():
            if x_email_rank: progress_bar_ngram.update(1)
            if new_ngram in all_ngrams:
                if not x_email_rank:
                    new_diff = len((new_data['bad_set'] - curr['bad_set']) & total_bad_set) / len(total_bad_set)
                else:
                    new_diff = sum([v for k,v in email_rank_bad.items() if k in (new_data['bad_set'] - curr['bad_set']) & total_bad_set])
                if new_diff > cur_diff:
                    curr_ngram = new_ngram
                    cur_diff = new_diff
                if cur_diff <= min_diff:
                    all_ngrams.remove(new_ngram)

        if cur_diff <= min_diff:
            break
        if sum([len(x) for x in curr['ngram_list']]) + len(curr_ngram) > max_len:
            break

        progress_bad_cov = len((calc_bad[curr_ngram]['bad_set'] - curr['bad_set']) & total_bad_set)
        progress_bar.update(progress_bad_cov)

        curr['bad_set'] |= calc_bad[curr_ngram]['bad_set']
        curr['ngram_list'].append(curr_ngram)
        all_ngrams.remove(curr_ngram)

        print(f"{curr['ngram_list']}\n Bad: {round(len(curr['bad_set']) / len(total_bad_set) * 100, 1)}%; Dif. in bads: {round(cur_diff * 100, 1)}%; Length: {sum([len(x) for x in curr['ngram_list']])}\n")

    return curr

x_print = True  # debug print if True

def dict_group_ngrams_withminus(df, calc_good, total_amount_clusters, ngrams_init=[[]], ngrams_last=[], req_good_set=set(), minus_bad_set=set(), try_max=8, max_len=1500,
                                target_good_cover=0.95, cut_bad_cover=0.5, step_bad_cover=0.1, target_good_inc=2, user_comp_imp=0, x_user_comp=False, x_tfidf=True):
    timer = defaultdict(int)
    calc_good = calc_good.copy()

    groups_good = {}

    union_good_set = set.union(*(value['good_set'] for value in calc_good.values())) # achievable goods
    union_bad_set = set.union(*(value['bad_set'] for value in calc_good.values())) - minus_bad_set # achievable bads

    progress_bar = tqdm(range(0, len(union_bad_set)))

    i = 0 # group number

    total_len = 0
    try_num = 0
    curr_cut_bad_cover = cut_bad_cover
    final_cover_good = 1
    forb_ngrams = set()

    email_rank_good = email_ranking(df, calc_good, set_name='good_set')
#    email_rank_bad = email_ranking(calc_good, set_name='bad_set')
    user_comb = len(user_unreached(df, union_good_set, set()))
    df_dict = df[['user_id']].to_dict('index')

    if ngrams_init != [[]]: # the ability to start collecting not from the first group
        for ngrams in ngrams_init:
            curr = {
                'ngram_list': [],
                'good_set': set(),
                'bad_set': set(),
            }
            for ngram in ngrams:
                curr['ngram_list'].append(ngram)
                curr['good_set'] |= calc_good[ngram]['good_set']
                curr['bad_set'] |= calc_good[ngram]['bad_set']
            group_len = sum([len(x) for x in curr['ngram_list']])

            total_len += group_len
            progress_bad_cov = len(union_bad_set - curr['bad_set'])
            progress_bar.update(progress_bad_cov)
            union_bad_set &= curr['bad_set']
            groups_good[i] = curr.copy()
            i += 1

    while True: # group building loop
        if x_print: print('Iteration №', try_num)
        try_num += 1
        curr = {
            'ngram_list': [],
            'good_set': set(),
            'bad_set': set(),
        }
        cover_prev = {'good': 0, 'bad': 0, 'inter': 0}
        curr_req_good_set = set()
        curr_req_good_set |= req_good_set # First cover the required part using only ngrams from required emails
        group_len = 0
        forb_ngrams = set()

        if ngrams_last != []:
            for ngram in ngrams_last:
                curr['ngram_list'].append(ngram)
                curr['good_set'] |= calc_good[ngram]['good_set']
                curr['bad_set'] |= calc_good[ngram]['bad_set']
            group_len = sum([len(x) for x in curr['ngram_list']])
            curr_req_good_set -= curr['good_set']
            ngrams_last = []

        while True:
            if i == 0:
                total_good_set = union_good_set
            else:
                total_good_set = set.intersection(*(value['good_set'] for value in groups_good.values())) # do not take into account the coverage+ that will be lost after crossing groups
            score_dict = {}
            tici = time.time() ## time reference point
            len_union_bad_set = len(union_bad_set)
            if user_comp_imp > 0:
                tic = time.time()
                user_dict = user_unreached(df, union_good_set, curr['good_set'] & total_good_set) # not yet covered user/company matchings # INTERSECT WITH PREVIOUS GROUP
                timer['user_dict'] += time.time()-tic
#                if x_user_comp and len(user_company_dict) == 0: # for a narrow query covering only user/company matchings
#                    break
            progress_bar_ngram = tqdm(range(0, len(calc_good)))
            bad_acc = (len(curr['good_set'] & total_good_set)/len(total_good_set)) - ((1/(1 - curr_cut_bad_cover)) * (len(curr['bad_set'] & union_bad_set) / len(union_bad_set))) # what percentage of coverage- was previously saved relative to the percentage of coverage+
            #print(bad_acc)
            for ngram_j, value_j in calc_good.items():
                progress_bar_ngram.update(1)
                tic = time.time()
                if ngram_j in forb_ngrams: ### IDEA - time saving on very bad ones
                    timer['forb_ngrams'] += time.time()-tic
                    continue
                timer['forb_ngrams'] += time.time()-tic
                tic = time.time()
                good_set_diff = (value_j['good_set'] - curr['good_set']) & total_good_set # how much coverage+ will be added by the ngram # INTERSECT WITH PREVIOUS GROUP
                timer['good_set_diff'] += time.time()-tic
                if len(good_set_diff) >= max(1, target_good_inc-i): # new ngram must add more than 1 email to coverage+
                    if curr_req_good_set:
                        if len(good_set_diff & curr_req_good_set) == 0: # first cover the required set using only ngrams from required emails
                            continue
                    # ngram should do more good than harm
                    tic = time.time()
                    bad_set_diff = (value_j['bad_set'] - curr['bad_set']) & union_bad_set
                    benifit_good = len(good_set_diff)/len(total_good_set)
                    benifit_bad = (1/(1 - curr_cut_bad_cover)) * len(bad_set_diff) / len(union_bad_set)
                    timer['bad_set_diff_1'] += time.time()-tic
                    #if len(good_set_diff)/len(total_good_set) < (1/(1 - curr_cut_bad_cover)) * len(bad_set_diff) / len(union_bad_set) - bad_acc:
                    if benifit_good < benifit_bad - bad_acc:
                        continue
                    tic = time.time()
                    ## Takes a long time
                    combined_bad_set = curr['bad_set'] | value_j['bad_set'] # bed set that will be on the next iteration
                    timer['combined_bad_set'] += time.time()-tic
                    tic = time.time()
                    bad_set_diff = union_bad_set - combined_bad_set
                    timer['bad_set_diff'] += time.time()-tic
                    ## Takes a long time
                    if len(bad_set_diff) > len_union_bad_set * curr_cut_bad_cover:
#                        if len(curr['ngram_list']) < target_group_len:
                            tic = time.time()
                            if x_tfidf:
                                score_dict[ngram_j] = sum([v for k,v in email_rank_good.items() if k in good_set_diff]) # bonus for uniqueness and length of coverage+
                            else:
                                score_dict[ngram_j] = len(good_set_diff)
                            timer['score_dict_good'] += time.time()-tic
                            # the query should to return one letter for each user/company pair
                            # list of unreached user/company pairs
                            if user_comp_imp > 0:
                                tic = time.time()
                                if user_dict:
                                    user_dict_copy = user_dict.copy()
                                    df_dict_filtered = {k: v for k,v in df_dict.items() if k in good_set_diff}
                                    # bonus only for first entry...
                                    for row in df_dict_filtered.values():
                                        user_key = row['user_id']
                                        if user_key in user_dict_copy:
                                            score_dict[ngram_j] = score_dict[ngram_j] + user_comp_imp * (1 + 0.5*i) # 57 = 1% гуд сета, потом полтора ### проверить на выбросы
                                            del user_dict_copy[user_key]
                                timer['user_score'] += time.time()-tic
                    else: ### IDEA - time saving on very bad ones
                        tic = time.time()
                        forb_ngrams.add(ngram_j)
                        timer['forb_ngrams'] += time.time()-tic
                else: ### IDEA - time saving on very bad ones
                    tic = time.time()
                    forb_ngrams.add(ngram_j)
                    timer['forb_ngrams'] += time.time()-tic

            if not score_dict: # break the loop if there are no ngrams left adding coverage+
                break
            # add to group
            tic = time.time()
            best_ngram = max(score_dict, key=score_dict.get)
            ### idea - if ngrams are equal, choose an ngram that covers more of the good ones - maximum overlap for reliability of hitting the query
            best_dict = {k: len(calc_good[k]['good_set']) for k,v in score_dict.items() if v == score_dict[best_ngram]} # len(calc_good[k]['good_set'])/total_amount_clusters['good_types'] - len(calc_good[k]['bad_set'])/total_amount_clusters['bad_types']
            #print(best_dict, ' len forb_ngrams', len(forb_ngrams)) #({k:v for k,v in score_dict.items() if v == score_dict[best_ngram]})
            best_ngram = max(best_dict, key=best_dict.get)
            ###
            timer['best_ngram'] += time.time()-tic
            if x_print: print('time of iter:', round(time.time()-tici, 2), 'len of score_dict', len(score_dict),
#                              'filtered', len(score_dict_filtered),
                              'added good', calc_good[best_ngram]['good_set'] - curr['good_set'])

            if total_len+group_len+sum([len(x) for x in best_ngram]) > max_len: # break the loop if too long group
                break

            for ngram_big in curr['ngram_list']: # removal of ngrams that are part of more general ngrams
                if best_ngram+' ' in ngram_big or ' '+best_ngram in ngram_big:
                    if x_print: print('ngram "', ngram_big, '" was removed by "', best_ngram, '"')
                    curr['ngram_list'].remove(ngram_big)


            curr['ngram_list'].append(best_ngram)
            curr['good_set'] |= calc_good[best_ngram]['good_set']
            curr['bad_set'] |= calc_good[best_ngram]['bad_set']
            group_len = sum([len(x) for x in curr['ngram_list']])

            if curr_req_good_set:
                curr_req_good_set -= curr['good_set']
                if len(curr_req_good_set) == 0:
                    print('--- Requied good set was cowered ---')

            if x_print:
                print('group №:', i, 'target_cut:', curr_cut_bad_cover, 'len:', group_len, '(', total_len+group_len, ')',
                              'cover good:', round(len(curr['good_set'])/total_amount_clusters['good_types'], 4), '(+', round(len(curr['good_set'])/total_amount_clusters['good_types']-cover_prev['good'], 4), ')',
                              'cover bad:', round(len(curr['bad_set'])/total_amount_clusters['bad_types'], 4), '(+', round(len(curr['bad_set'])/total_amount_clusters['bad_types']-cover_prev['bad'], 4), ')',
                              'cover bad inter:', round(len(union_bad_set & curr['bad_set'])/len_union_bad_set, 4), '(+', round(len(union_bad_set & curr['bad_set'])/len_union_bad_set-cover_prev['inter'], 4), ')',
                              'unreached combination len:', len(user_unreached(df, union_good_set, curr['good_set'] & total_good_set)), '//', user_comb, 'unreached requied:', len(curr_req_good_set), '//', len(req_good_set),  ### ПЕРЕСЕЧЬ С ПРЕДЫДУЩЕЙ ГРУППОЙ???
                              '\n', len(curr['ngram_list']), curr['ngram_list'])
                cover_prev['good'] = round(len(curr['good_set'])/total_amount_clusters['good_types'], 4)
                cover_prev['bad'] = round(len(curr['bad_set'])/total_amount_clusters['bad_types'], 4)
                cover_prev['inter'] = round(len(union_bad_set & curr['bad_set'])/len_union_bad_set, 4)
            if x_print: print(timer.items())

        if x_print: print(timer.items())
        if len(set.intersection(*(value['good_set'] for value in groups_good.values()), curr['good_set'])) / total_amount_clusters['good_types'] > target_good_cover:
            if total_len+group_len <= max_len: # only groups with the target length are saved
                progress_bad_cov = len(union_bad_set - curr['bad_set'])
                progress_bar.update(progress_bad_cov)
                union_bad_set &= curr['bad_set']
                if not progress_bad_cov: # break the loop if the new group does not cut anything off from the general coverage-
                    break

                total_len += group_len
                groups_good[i] = curr.copy()
                curr_cut_bad_cover = cut_bad_cover
                final_cover_good = len(set.intersection(*(value['good_set'] for value in groups_good.values()))) / total_amount_clusters['good_types']
                if x_print: print('+++Continue. Cover bad by groups:', round(len(union_bad_set) / total_amount_clusters['bad_types'], 4), 'Cover good by groups:', round(final_cover_good, 4), 'Groups length:', sum([sum([len(y) for y in x['ngram_list']]) for x in groups_good.values()]), '\n')
                if not len(union_bad_set): # break the loop if the coverage- is over
                    break
                #for ngram in curr['ngram_list']: all_ngrams.append(ngram)

                i += 1 # a new group
            else:
                if x_print: print('Over len. Cover bad by groups:', round(len(union_bad_set & curr['bad_set']) / total_amount_clusters['bad_types'], 4), 'Cover good by groups:', round(final_cover_good*len(curr['good_set'])/total_amount_clusters['good_types'], 4), 'Groups length:', total_len+group_len, '\n')
                curr_cut_bad_cover -= step_bad_cover
        else:
            if x_print: print('Not enouth. Over Cover bad by groups:', round(len(union_bad_set & curr['bad_set']) / total_amount_clusters['bad_types'], 4), 'Cover good by groups:', round(final_cover_good*len(curr['good_set'])/total_amount_clusters['good_types'], 4), 'Groups length:', total_len+group_len, '\n')
            curr_cut_bad_cover -= step_bad_cover

        if curr_cut_bad_cover < step_bad_cover: # break the loop if the new group does not cut off anything from the overall coverage-
            break

        if try_num > try_max:
            break

        if group_len > max_len * 0.9:
            break

    for group in groups_good.values():
        group['good_types_cover'] = len(group['good_set']) / total_amount_clusters['good_types'] # final coverage+ %
        group['bad_types_cover'] = len(group['bad_set']) / total_amount_clusters['bad_types'] # final coverage- %

    if x_print: print(total_len, timer.items())

    return groups_good, total_len, final_cover_good, len(union_bad_set) / total_amount_clusters['bad_types']

def groups_to_query(groups_good, base_query, total_amount_clusters):
    total_good_set = set()
    total_bad_set = set()
    query = base_query + ' '

    total_good_set |= groups_good[0]['good_set']
    total_bad_set |= groups_good[0]['bad_set']
    for i in range(0, int(len(groups_good))):
        total_good_set &= groups_good[i]['good_set']
        total_bad_set &= groups_good[i]['bad_set']
        query = query + '('
        for ngram in groups_good[i]['ngram_list']:
            if ' ' in ngram:
                query = query + '\"' + ngram + '\"|'
            else:
                query = query + ngram + '|'
        query = query[:-1] + ') '
        print(f"Total cover of {i} groups: len {len(query)}, good {len(total_good_set) / total_amount_clusters['good_types']}, bad {len(total_bad_set) / total_amount_clusters['bad_types']}")
    query = query[:-1]
    query = query + ' -wisepal'

    return query, total_good_set, total_bad_set

def minus_to_query(group_minus, base_query):
    query = base_query + ' -'
    for ngram in group_minus['ngram_list']:
        if ' ' in ngram:
            query = query + '\"' + ngram + '\" '
        else:
            query = query + ngram + ' '
        query = query[:-1]+' -'

    query = query[:-2]

    return query

# final union function
def pipeline_of_query_creating(path_params, ngram_params, minus_params, group_params, base_query, x_test):
    ### fix df ###
    print('### Reading df ###')
    df = read_and_fix_df(path_params['df'], x_test=x_test)

    ### Generate ngram dict ###
    print('### Generate ngram dict ###')

    total_amount_clusters = count_total_amount_clusters(df)
    print('border_good:', int(total_amount_clusters['good_types'] * ngram_params['border_good_cover'] + 1), '/', total_amount_clusters['good_types'])
    print(' border_bad:', int(total_amount_clusters['bad_types'] * ngram_params['border_bad_cover'] + 1), '/', total_amount_clusters['bad_types'])

    calc_good = calc_bad = {}
    if path_params['wb_calc'] == False:
        try:
            with open(path_params['calc_good'], 'rb') as f:
                calc_good = pickle.load(f)

            with open(path_params['calc_bad'], 'rb') as f:
                calc_bad = pickle.load(f)
        except Exception as e:
            print(e)

    if len(calc_good) == 0:
        gc.collect()
        tic = time.time()
        calc_good, calc_bad = dict_by_clusters(df, total_amount_clusters, n_grams=ngram_params['n_grams'],
                                               good_types=ngram_params['good_types'], bad_types=ngram_params['bad_types'], neutral_types=ngram_params['neutral_types'],
                                               trash_ngrams=ngram_params['trash_ngrams'], trash_words_in_ngrams=ngram_params['trash_words_in_ngrams'],
                                               forbidden_words=ngram_params['forbidden_words'], ngrams_in_pdf=ngram_params['ngrams_in_pdf'],
                                               border_good_cover=ngram_params['border_good_cover'], border_bad_cover=ngram_params['border_bad_cover'])

        print(f'###### Time of dict_by_clusters ', time.time()-tic, ' Len of calc_good ', len(calc_good), ' ###\n')
        gc.collect()

        with open(path_params['calc_good'], 'wb') as f:
            pickle.dump(calc_good, f)

        with open(path_params['calc_bad'], 'wb') as f:
            pickle.dump(calc_bad, f)

    ### Generate minus words ###
    print('### Generate minus words ###')
    group_minus = {}
    if path_params['wb_group_minus'] == False:
        try:
            with open(path_params['group_minus'], 'rb') as f:
                groups = pickle.load(f)
        except Exception as e:
            print(e)

    if len(group_minus) == 0:
        tic = time.time()
        group_minus = dict_minus_ngram(df, calc_bad, total_amount_clusters, set.union(*(value['bad_set'] for value in calc_good.values())),
                                    max_len=minus_params['max_len'], min_diff=minus_params['min_diff'], x_email_rank=minus_params['x_email_rank'], email_only=minus_params['email_only'])
        rest_minus_len = minus_params['max_len'] - sum([len(x) for x in group_minus['ngram_list']])
        print(f'###### Time of dict_minus_ngram ', time.time()-tic, ' Rest len of group_minus ', rest_minus_len, ' ###\n')

        with open(path_params['group_minus'], 'wb') as f:
            pickle.dump(group_minus, f)

    ### Generate ngram groups ###
    print('### Generate ngram groups ###')
    groups = {}
    if path_params['wb_groups'] == False:
        try:
            with open(path_params['groups'], 'rb') as f:
                groups = pickle.load(f)
        except Exception as e:
            print(e)

    if len(groups) == 0:
        tic = time.time()
        groups, total_len, good_cover, bad_cover = dict_group_ngrams_withminus(df, calc_good, total_amount_clusters,
                                                                               ngrams_init=group_params['ngrams_init'], ngrams_last=group_params['ngrams_last'],
                                                                               req_good_set=group_params['req_good_set'], minus_bad_set=group_minus['bad_set'],
                                                                               try_max=group_params['try_max'], max_len=group_params['max_len']+rest_minus_len,
                                                                               target_good_cover=group_params['target_good_cover'], cut_bad_cover=group_params['cut_bad_cover'], step_bad_cover=group_params['step_bad_cover'],
                                                                               target_good_inc=group_params['target_good_inc'], user_comp_imp=group_params['user_comp_imp'],
                                                                               x_user_comp=group_params['x_user_comp'], x_tfidf=group_params['x_tfidf'])
        print(f'###### Time of dict_group_ngrams_withminus ', time.time()-tic, ' ###\n')
        with open(path_params['groups'], 'wb') as f:
            pickle.dump(groups, f)

    ### Concatinate groups to query ###
    print('### Concatinate groups to query ###')
    query, total_good_set, total_bad_set = groups_to_query(groups, base_query, total_amount_clusters)
    print('Query with groups len: ', len(query))
    #print(query)

    query = minus_to_query(group_minus, query)
    print("Final bad cover:", len(total_bad_set - group_minus['bad_set']) / total_amount_clusters['bad_types'])
    print(f'###### Final query len: ', len(query), ' ###\n')
    print(query)

    ### Outliers ###
    print(f'\n### Outliers ###')
    union_good_set = set.union(*(value['good_set'] for value in calc_good.values()))
    out_good_set = union_good_set - total_good_set
    print('Total good set len:', len(union_good_set), ' Outliers good set len:', len(out_good_set), ' outliers:', out_good_set)

    pd.options.display.max_colwidth = 250
    for link_id in out_good_set:
        print(df.loc[df['link_id'] == link_id, ['user_id', 'context_type', 'link']].to_dict('records'))

    return df, calc_good, calc_bad, group_minus, groups, query, out_good_set

# input data
path_params = {'df': r'/Users/shahardubiner/Documents/WisePal/ssn_df_grouped_fixman v3.json',
                'wb_calc': True,  # rewrite
                'calc_good': r'ssn_test_calc_good_v3.pkl',
                'calc_bad': r'ssn_test_calc_bad_v3.pkl',
                'wb_group_minus': True,  # rewrite
                'group_minus': r'ssn_test_group_minus_v3.pkl',
                'wb_groups': True,  # rewrite
                'groups': r'ssn_test_groups_v3.pkl'
}


ngram_params = {'n_grams': [1,4],
                'good_types': ['good'], # class names
                'bad_types': ['bad'], # class names
                'neutral_types': ['neutral'],
                'forbidden_words': forbidden_words, # forbidden by Google
                'trash_ngrams': ['45am', '3.2', '2.3', '2013,', '01,', '26,', '43pm', '26th', '1967,', '45pm', '22nd', '35pm', '2.75', '59pm', '59am', '00pm', '00am'], # remove odd
                'trash_words_in_ngrams': ['gmail'], # remove user's email adresses
                'ngrams_in_pdf': ["twitter com", "greg hdcomptoninsurance com"], # forbidden for use like minus word, becouse they included in img-alt, href or pdf
                'border_good_cover': 0.001, # 0.0004, # down border
                'border_bad_cover': 0.003 # 0.001 # down border
}

minus_params = {'max_len': 500,
                'min_diff': 0.001,
                'x_email_rank': False,
                'email_only': True # safty mode when minus words are emeil adresses and domains only
                }

group_params = {'ngrams_init': [[]], # if need to start from second or third group
                'ngrams_last': [], # if need to continue group
                'req_good_set': set(), #set(df.loc[df['importance'] == 1, 'link_id'].to_list()) # list of emails required for coverage - example: 1 per user/company pair
                #minus_bad_set=group_minus['bad_set']  # ignored coverage- filtered by minus words
                'try_max': 1, # number of tries to build groups in total
                'max_len': 1650, # net length of ngrams; for home maximum length of query ~1950 including minus words
                'target_good_cover': 0.95, # 0.9995 # target coverage+
                'cut_bad_cover': 0.35, # 0.4 # target cutting of the coverage- for each group
                'step_bad_cover': 0.10, # step of target bad cutting
                'target_good_inc': 2, # lower limit on the increase in coverage+, for example, add at least 3 letters+ per ngram in the first group, 2 in the second...
                'user_comp_imp': 0, #int(total_amount_clusters['good_types'] * 0.01) # importance of matching coverage+ as ~1% good coverage for 1 group, for subsequent * 1.5 // time of iter x2
                'x_user_comp': False, # mode in which the group is completed when all user/company matchings are covered
                'x_tfidf': True # use tf-idf score // time of iter x2
                }

base_query = 'in:anywhere ("social security number"|"ssn"|"soc sec num"|"Social Security"|"Social Sec"|"Soc Sec"|"Soc Security Number"|"Social Security No"|"SS No"|"SS Num"|"SS Number"|"Social Security Administration"|"SSA"|"Social Sec Admin"|"Social Security Admin"|"Soc Sec Admin"|"Soc Security Administration"|"Soc Security Admin"|"SS Administration") '

x_test = True

df, calc_good, calc_bad, group_minus, groups, query, out_good_set = pipeline_of_query_creating(path_params, ngram_params, minus_params, group_params, base_query, x_test)

### Reading df ###
type
bad        9495
good        450
neutral      55
Name: count, dtype: int64
### Generate ngram dict ###
border_good: 1 / 409
 border_bad: 28 / 9209


100%|█████████▉| 9989/10000 [01:35<00:00, 101.34it/s]

Iteration: 9999, clusters_count size: 7328040, temp_dict size: 909121
Size in RAM - clusters_count: 5.370948381721973 Gb, temp_dict: 1.0965395495295525 Gb


100%|██████████| 909121/909121 [01:22<00:00, 11085.35it/s]  


###### Time of dict_by_clusters  199.85746693611145  Len of calc_good  857546  ###



OSError: [Errno 30] Read-only file system: 'ssn_test_calc_good_v3.pkl'